# SQL Exercise Solutions - 02 SQL Concepts

## Introduction

This notebook contains **complete solutions** to all exercise questions from "2i SQL Exercise Questions.ipynb". Each solution includes:

- ✅ Complete SQL query in **Snowflake SQL**
- 📝 Step-by-step explanation
- 💡 Key concepts demonstrated
- 🔄 SQL Server notes where syntax differs
- ⚠️ Important considerations and edge cases

**Database:** Solutions are written for **Snowflake**. SQL Server differences are noted where applicable.

**Dataset:** All solutions use tables from `CETPA_DB.PUBLIC` schema.

---

## Table Schema Reference

```sql
-- CUSTOMERS
CUSTOMER_ID (VARCHAR)
CUSTOMER_UNIQUE_ID (VARCHAR)
CUSTOMER_ZIP_CODE_PREFIX (NUMBER)
CUSTOMER_CITY (VARCHAR)
CUSTOMER_STATE (VARCHAR)

-- geolocation
GEOLOCATION_ZIP_CODE_PREFIX (NUMBER)
GEOLOCATION_LAT (NUMBER)
GEOLOCATION_LNG (NUMBER)
GEOLOCATION_CITY (VARCHAR)
GEOLOCATION_STATE (VARCHAR)

-- orders
ORDER_ID (VARCHAR)
CUSTOMER_ID (VARCHAR)
ORDER_STATUS (VARCHAR)
ORDER_PURCHASE_TIMESTAMP (TIMESTAMP_NTZ)
ORDER_APPROVED_AT (TIMESTAMP_NTZ)
ORDER_DELIVERED_CARRIER_DATE (TIMESTAMP_NTZ)
ORDER_DELIVERED_CUSTOMER_DATE (TIMESTAMP_NTZ)
ORDER_ESTIMATED_DELIVERY_DATE (TIMESTAMP_NTZ)

-- order_items
ORDER_ID (VARCHAR)
ORDER_ITEM_ID (NUMBER)
PRODUCT_ID (VARCHAR)
SELLER_ID (VARCHAR)
SHIPPING_LIMIT_DATE (TIMESTAMP_NTZ)
PRICE (NUMBER(38,2))
FREIGHT_VALUE (NUMBER(38,2))

-- order_payments
ORDER_ID (VARCHAR)
PAYMENT_SEQUENTIAL (NUMBER)
PAYMENT_TYPE (VARCHAR)
PAYMENT_INSTALLMENTS (NUMBER)
PAYMENT_VALUE (NUMBER(38,2))

-- order_reviews
REVIEW_ID (VARCHAR)
ORDER_ID (VARCHAR)
REVIEW_SCORE (NUMBER)
REVIEW_COMMENT_TITLE (VARCHAR)
REVIEW_COMMENT_MESSAGE (VARCHAR)
REVIEW_CREATION_DATE (TIMESTAMP_NTZ)
REVIEW_ANSWER_TIMESTAMP (TIMESTAMP_NTZ)

-- products
PRODUCT_ID (VARCHAR)
PRODUCT_CATEGORY_NAME (VARCHAR)
PRODUCT_NAME_LENGHT (NUMBER)
PRODUCT_DESCRIPTION_LENGHT (NUMBER)
PRODUCT_PHOTOS_QTY (NUMBER)
PRODUCT_WEIGHT_G (NUMBER)
PRODUCT_LENGTH_CM (NUMBER)
PRODUCT_HEIGHT_CM (NUMBER)
PRODUCT_WIDTH_CM (NUMBER)

-- product_category_name_translation
C1 (VARCHAR)  -- Portuguese category name
C2 (VARCHAR)  -- English category name

-- sellers
SELLER_ID (VARCHAR)
SELLER_ZIP_CODE_PREFIX (NUMBER)
SELLER_CITY (VARCHAR)
SELLER_STATE (VARCHAR)
```

---

## Section 1: Basic Joins (🟢 Beginner)



### Solution 1.1: Customer Orders

**Problem:** Show all customers with their order IDs and order dates. Include customers who have never placed an order.

**Approach:**
- Use LEFT JOIN to preserve all customers
- Join CUSTOMERS with orders on CUSTOMER_ID
- Customers without orders will have NULL values for order columns

**Key Concepts:**
- LEFT JOIN preserves all rows from the left table
- NULL values indicate missing relationships


In [1]:
-- Solution 1.1: Customer Orders
SELECT 
    c.CUSTOMER_ID,
    c.CUSTOMER_UNIQUE_ID,
    o.ORDER_ID,
    o.ORDER_PURCHASE_TIMESTAMP
FROM CETPA_DB.PUBLIC.CUSTOMERS c
LEFT JOIN CETPA_DB.PUBLIC.orders o 
    ON c.CUSTOMER_ID = o.CUSTOMER_ID
ORDER BY c.CUSTOMER_ID, o.ORDER_PURCHASE_TIMESTAMP;

-- SQL Server Note: Syntax is identical in SQL Server


SyntaxError: invalid syntax (3394921375.py, line 1)

### Solution 1.2: Order Details with Products

**Problem:** Show all order items with their product information.

**Approach:**
- Join order_items with products on PRODUCT_ID
- Use INNER JOIN since we want only order items with valid products

**Key Concepts:**
- INNER JOIN returns only matching records
- One-to-many relationship: one product can appear in many order items


In [ ]:
-- Solution 1.2: Order Details with Products
SELECT 
    oi.ORDER_ID,
    oi.ORDER_ITEM_ID,
    oi.PRODUCT_ID,
    p.PRODUCT_CATEGORY_NAME,
    oi.PRICE,
    oi.FREIGHT_VALUE
FROM CETPA_DB.PUBLIC.order_items oi
INNER JOIN CETPA_DB.PUBLIC.products p 
    ON oi.PRODUCT_ID = p.PRODUCT_ID
ORDER BY oi.ORDER_ID, oi.ORDER_ITEM_ID;

-- SQL Server Note: Syntax is identical


### Solution 1.3: Orders Without Items

**Problem:** Find all orders that have no order items.

**Approach:**
- Use LEFT JOIN from orders to order_items
- Filter for NULL ORDER_ITEM_ID to find orders without items

**Key Concepts:**
- LEFT JOIN + WHERE NULL pattern finds missing relationships
- This identifies data quality issues (orphaned orders)


In [ ]:
-- Solution 1.3: Orders Without Items
SELECT 
    o.ORDER_ID,
    o.ORDER_STATUS,
    o.ORDER_PURCHASE_TIMESTAMP
FROM CETPA_DB.PUBLIC.orders o
LEFT JOIN CETPA_DB.PUBLIC.order_items oi 
    ON o.ORDER_ID = oi.ORDER_ID
WHERE oi.ORDER_ITEM_ID IS NULL
ORDER BY o.ORDER_PURCHASE_TIMESTAMP;

-- SQL Server Note: Syntax is identical


---

## Section 2: Multiple Joins (🟡 Intermediate)

### Solution 2.1: Complete Order Information

**Problem:** Create a comprehensive order report with all related information.

**Approach:**
- Multiple joins: orders → customers, orders → order_items → products → category translation, orders → order_payments
- Handle one-to-many relationships (one order can have multiple items and payments)
- Use LEFT JOINs where relationships might be missing

**Key Concepts:**
- Multiple table joins
- Handling one-to-many relationships
- LEFT JOIN for optional relationships


In [ ]:
-- Solution 2.1: Complete Order Information
SELECT 
    o.ORDER_ID,
    c.CUSTOMER_CITY,
    c.CUSTOMER_STATE,
    o.ORDER_STATUS,
    o.ORDER_PURCHASE_TIMESTAMP,
    COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME) AS PRODUCT_CATEGORY_NAME_ENGLISH,
    oi.PRICE,
    op.PAYMENT_TYPE,
    op.PAYMENT_VALUE
FROM CETPA_DB.PUBLIC.orders o
INNER JOIN CETPA_DB.PUBLIC.CUSTOMERS c 
    ON o.CUSTOMER_ID = c.CUSTOMER_ID
INNER JOIN CETPA_DB.PUBLIC.order_items oi 
    ON o.ORDER_ID = oi.ORDER_ID
INNER JOIN CETPA_DB.PUBLIC.products p 
    ON oi.PRODUCT_ID = p.PRODUCT_ID
LEFT JOIN CETPA_DB.PUBLIC.product_category_name_translation trans 
    ON p.PRODUCT_CATEGORY_NAME = trans.C1
LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
    ON o.ORDER_ID = op.ORDER_ID
ORDER BY o.ORDER_ID, oi.ORDER_ITEM_ID, op.PAYMENT_SEQUENTIAL;

-- Note: This query may return multiple rows per order if an order has multiple items or payments
-- SQL Server Note: Syntax is identical


### Solution 2.2: Customer Order Summary

**Problem:** Show all customers with their order statistics, including customers with no orders.

**Approach:**
- Use LEFT JOIN to include all customers
- Aggregate order and payment data
- Use COALESCE to handle NULLs from customers without orders

**Key Concepts:**
- LEFT JOIN preserves all customers
- Aggregation with GROUP BY
- NULL handling with COALESCE


In [ ]:
-- Solution 2.2: Customer Order Summary
SELECT 
    c.CUSTOMER_ID,
    c.CUSTOMER_UNIQUE_ID,
    c.CUSTOMER_CITY,
    c.CUSTOMER_STATE,
    COUNT(DISTINCT o.ORDER_ID) AS TOTAL_ORDERS,
    COALESCE(SUM(op.PAYMENT_VALUE), 0) AS TOTAL_AMOUNT_SPENT,
    COALESCE(AVG(op.PAYMENT_VALUE), 0) AS AVERAGE_ORDER_VALUE
FROM CETPA_DB.PUBLIC.CUSTOMERS c
LEFT JOIN CETPA_DB.PUBLIC.orders o 
    ON c.CUSTOMER_ID = o.CUSTOMER_ID
LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
    ON o.ORDER_ID = op.ORDER_ID
GROUP BY 
    c.CUSTOMER_ID,
    c.CUSTOMER_UNIQUE_ID,
    c.CUSTOMER_CITY,
    c.CUSTOMER_STATE
ORDER BY TOTAL_AMOUNT_SPENT DESC;

-- SQL Server Note: Syntax is identical


### Solution 2.3: Seller Performance

**Problem:** Show seller performance metrics.

**Approach:**
- Join sellers with order_items
- Aggregate metrics per seller
- Calculate revenue as sum of price + freight_value

**Key Concepts:**
- Aggregation with multiple metrics
- COUNT DISTINCT for unique counts
- Revenue calculation


In [ ]:
-- Solution 2.3: Seller Performance
SELECT 
    s.SELLER_ID,
    s.SELLER_CITY,
    s.SELLER_STATE,
    COUNT(DISTINCT oi.ORDER_ID) AS UNIQUE_ORDERS,
    COALESCE(SUM(oi.PRICE + oi.FREIGHT_VALUE), 0) AS TOTAL_REVENUE,
    COALESCE(AVG(oi.PRICE + oi.FREIGHT_VALUE), 0) AS AVERAGE_ORDER_VALUE,
    COUNT(DISTINCT oi.PRODUCT_ID) AS UNIQUE_PRODUCTS_SOLD
FROM CETPA_DB.PUBLIC.sellers s
LEFT JOIN CETPA_DB.PUBLIC.order_items oi 
    ON s.SELLER_ID = oi.SELLER_ID
GROUP BY 
    s.SELLER_ID,
    s.SELLER_CITY,
    s.SELLER_STATE
ORDER BY TOTAL_REVENUE DESC;

-- SQL Server Note: Syntax is identical


---

## Section 3: Handling NULLs (🟡 Intermediate)

### Solution 3.1: Safe Revenue Calculation

**Problem:** Calculate total revenue per order, handling NULLs properly.

**Approach:**
- Use COALESCE to replace NULLs with 0 before calculation
- Sum price + freight_value for each order
- Group by order to get total per order

**Key Concepts:**
- COALESCE for NULL handling
- Safe arithmetic operations
- Aggregation with NULL handling


In [ ]:
-- Solution 3.1: Safe Revenue Calculation
SELECT 
    o.ORDER_ID,
    o.ORDER_STATUS,
    COALESCE(SUM(COALESCE(oi.PRICE, 0) + COALESCE(oi.FREIGHT_VALUE, 0)), 0) AS CALCULATED_REVENUE
FROM CETPA_DB.PUBLIC.orders o
LEFT JOIN CETPA_DB.PUBLIC.order_items oi 
    ON o.ORDER_ID = oi.ORDER_ID
GROUP BY 
    o.ORDER_ID,
    o.ORDER_STATUS
ORDER BY CALCULATED_REVENUE DESC;

-- Alternative approach using COALESCE on the entire expression:
-- COALESCE(SUM(oi.PRICE + oi.FREIGHT_VALUE), 0) AS CALCULATED_REVENUE
-- But the above is safer as it handles NULLs in individual columns

-- SQL Server Note: 
-- SQL Server supports ISNULL() as alternative: ISNULL(SUM(oi.PRICE + oi.FREIGHT_VALUE), 0)
-- But COALESCE is more portable


### Solution 3.2: Customer Display Names

**Problem:** Create customer display names with NULL handling.

**Approach:**
- Use COALESCE to provide defaults for NULL city and state
- String concatenation with || operator
- Format: "Customer [ID] from [CITY], [STATE]"

**Key Concepts:**
- String concatenation
- COALESCE for text defaults
- NULL handling in string operations


In [ ]:
-- Solution 3.2: Customer Display Names
SELECT 
    CUSTOMER_ID,
    'Customer ' || CUSTOMER_ID || ' from ' || 
    COALESCE(CUSTOMER_CITY, 'Unknown City') || ', ' || 
    COALESCE(CUSTOMER_STATE, 'Unknown State') AS CUSTOMER_DISPLAY_NAME
FROM CETPA_DB.PUBLIC.CUSTOMERS
ORDER BY CUSTOMER_ID;

-- SQL Server Note:
-- SQL Server uses + for string concatenation instead of ||
-- 'Customer ' + CUSTOMER_ID + ' from ' + 
-- ISNULL(CUSTOMER_CITY, 'Unknown City') + ', ' + 
-- ISNULL(CUSTOMER_STATE, 'Unknown State') AS CUSTOMER_DISPLAY_NAME
-- Or use CONCAT() function: CONCAT('Customer ', CUSTOMER_ID, ' from ', ...)


### Solution 3.3: Product Category Analysis

**Problem:** Analyze products by category, handling NULL categories.

**Approach:**
- Use COALESCE in GROUP BY to group NULL categories together
- Aggregate product metrics
- Handle NULLs in calculations

**Key Concepts:**
- COALESCE in GROUP BY
- Aggregation with NULL handling
- Calculating averages with NULLs


In [ ]:
-- Solution 3.3: Product Category Analysis
SELECT 
    COALESCE(p.PRODUCT_CATEGORY_NAME, 'Uncategorized') AS CATEGORY_NAME,
    COUNT(*) AS NUMBER_OF_PRODUCTS,
    AVG(COALESCE(p.PRODUCT_WEIGHT_G, 0)) AS AVG_PRODUCT_WEIGHT_G,
    AVG(
        COALESCE(p.PRODUCT_LENGTH_CM, 0) + 
        COALESCE(p.PRODUCT_WIDTH_CM, 0) + 
        COALESCE(p.PRODUCT_HEIGHT_CM, 0)
    ) AS AVG_PRODUCT_DIMENSIONS_CM
FROM CETPA_DB.PUBLIC.products p
GROUP BY COALESCE(p.PRODUCT_CATEGORY_NAME, 'Uncategorized')
ORDER BY NUMBER_OF_PRODUCTS DESC;

-- SQL Server Note: Syntax is identical


---

## Section 4: GROUP BY and Aggregations (🟡 Intermediate)

### Solution 4.1: Order Status Summary

**Problem:** Create a summary of orders by status.

**Approach:**
- Group by ORDER_STATUS
- Calculate multiple aggregate metrics
- Order by count descending

**Key Concepts:**
- GROUP BY with multiple aggregates
- COUNT, SUM, AVG, MIN, MAX functions


In [ ]:
-- Solution 4.1: Order Status Summary
SELECT 
    o.ORDER_STATUS,
    COUNT(*) AS COUNT_OF_ORDERS,
    COALESCE(SUM(op.PAYMENT_VALUE), 0) AS TOTAL_REVENUE,
    COALESCE(AVG(op.PAYMENT_VALUE), 0) AS AVERAGE_ORDER_VALUE,
    MIN(op.PAYMENT_VALUE) AS MINIMUM_ORDER_VALUE,
    MAX(op.PAYMENT_VALUE) AS MAXIMUM_ORDER_VALUE
FROM CETPA_DB.PUBLIC.orders o
LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
    ON o.ORDER_ID = op.ORDER_ID
GROUP BY o.ORDER_STATUS
ORDER BY COUNT_OF_ORDERS DESC;

-- SQL Server Note: Syntax is identical


### Solution 4.2: Top Customers by Revenue

**Problem:** Find the top 10 customers by total revenue.

**Approach:**
- Join customers with orders and order_payments
- Aggregate by customer
- Order by total revenue descending
- Limit to top 10

**Key Concepts:**
- Multiple table joins with aggregation
- LIMIT clause (TOP in SQL Server)
- Ranking customers by revenue


In [ ]:
-- Solution 4.2: Top Customers by Revenue
SELECT 
    c.CUSTOMER_ID,
    c.CUSTOMER_UNIQUE_ID,
    c.CUSTOMER_CITY,
    COUNT(DISTINCT o.ORDER_ID) AS TOTAL_ORDERS,
    COALESCE(SUM(op.PAYMENT_VALUE), 0) AS TOTAL_REVENUE,
    COALESCE(AVG(op.PAYMENT_VALUE), 0) AS AVERAGE_ORDER_VALUE
FROM CETPA_DB.PUBLIC.CUSTOMERS c
INNER JOIN CETPA_DB.PUBLIC.orders o 
    ON c.CUSTOMER_ID = o.CUSTOMER_ID
LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
    ON o.ORDER_ID = op.ORDER_ID
GROUP BY 
    c.CUSTOMER_ID,
    c.CUSTOMER_UNIQUE_ID,
    c.CUSTOMER_CITY
ORDER BY TOTAL_REVENUE DESC
LIMIT 10;

-- SQL Server Note:
-- Use TOP instead of LIMIT:
-- SELECT TOP 10 ...
-- ORDER BY TOTAL_REVENUE DESC;


### Solution 4.3: Monthly Sales Report

**Problem:** Create a monthly sales report.

**Approach:**
- Extract year and month from ORDER_PURCHASE_TIMESTAMP
- Group by year and month
- Aggregate order and revenue metrics

**Key Concepts:**
- Date extraction functions
- Grouping by date parts
- Time-based aggregations


In [ ]:
-- Solution 4.3: Monthly Sales Report
SELECT 
    EXTRACT(YEAR FROM o.ORDER_PURCHASE_TIMESTAMP) AS YEAR,
    EXTRACT(MONTH FROM o.ORDER_PURCHASE_TIMESTAMP) AS MONTH,
    COUNT(DISTINCT o.ORDER_ID) AS NUMBER_OF_ORDERS,
    COUNT(DISTINCT o.CUSTOMER_ID) AS UNIQUE_CUSTOMERS,
    COALESCE(SUM(op.PAYMENT_VALUE), 0) AS TOTAL_REVENUE,
    COALESCE(AVG(op.PAYMENT_VALUE), 0) AS AVERAGE_ORDER_VALUE
FROM CETPA_DB.PUBLIC.orders o
LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
    ON o.ORDER_ID = op.ORDER_ID
GROUP BY 
    EXTRACT(YEAR FROM o.ORDER_PURCHASE_TIMESTAMP),
    EXTRACT(MONTH FROM o.ORDER_PURCHASE_TIMESTAMP)
ORDER BY YEAR, MONTH;

-- SQL Server Note:
-- Use YEAR() and MONTH() functions instead:
-- YEAR(o.ORDER_PURCHASE_TIMESTAMP) AS YEAR,
-- MONTH(o.ORDER_PURCHASE_TIMESTAMP) AS MONTH,
-- GROUP BY YEAR(o.ORDER_PURCHASE_TIMESTAMP), MONTH(o.ORDER_PURCHASE_TIMESTAMP)


### Solution 4.4: Product Category Performance

**Problem:** Show product category performance with revenue filter.

**Approach:**
- Join products with order_items
- Join with category translation for English names
- Aggregate by category
- Filter with HAVING for revenue > 1000

**Key Concepts:**
- HAVING clause for filtering groups
- COALESCE for handling missing translations
- Revenue calculation from order items


In [ ]:
-- Solution 4.4: Product Category Performance
SELECT 
    COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME) AS CATEGORY_NAME_ENGLISH,
    COUNT(DISTINCT p.PRODUCT_ID) AS NUMBER_OF_UNIQUE_PRODUCTS,
    COALESCE(SUM(oi.PRICE + oi.FREIGHT_VALUE), 0) AS TOTAL_REVENUE,
    COALESCE(AVG(oi.PRICE), 0) AS AVERAGE_PRODUCT_PRICE
FROM CETPA_DB.PUBLIC.products p
INNER JOIN CETPA_DB.PUBLIC.order_items oi 
    ON p.PRODUCT_ID = oi.PRODUCT_ID
LEFT JOIN CETPA_DB.PUBLIC.product_category_name_translation trans 
    ON p.PRODUCT_CATEGORY_NAME = trans.C1
GROUP BY COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME)
HAVING COALESCE(SUM(oi.PRICE + oi.FREIGHT_VALUE), 0) > 1000
ORDER BY TOTAL_REVENUE DESC;

-- SQL Server Note: Syntax is identical


### Solution 4.5: Payment Type Analysis

**Problem:** Analyze payment types.

**Approach:**
- Aggregate from order_payments table
- Group by PAYMENT_TYPE
- Calculate payment statistics

**Key Concepts:**
- Simple aggregation
- COUNT DISTINCT for unique orders
- Payment analysis


In [ ]:
-- Solution 4.5: Payment Type Analysis
SELECT 
    PAYMENT_TYPE,
    COUNT(*) AS NUMBER_OF_PAYMENTS,
    SUM(PAYMENT_VALUE) AS TOTAL_PAYMENT_VALUE,
    AVG(PAYMENT_VALUE) AS AVERAGE_PAYMENT_VALUE,
    COUNT(DISTINCT ORDER_ID) AS NUMBER_OF_UNIQUE_ORDERS
FROM CETPA_DB.PUBLIC.order_payments
GROUP BY PAYMENT_TYPE
ORDER BY TOTAL_PAYMENT_VALUE DESC;

-- SQL Server Note: Syntax is identical


---

## Section 5: Window Functions (🟡 Intermediate to 🔴 Advanced)

### Solution 5.1: Order Ranking by Customer

**Problem:** Rank orders by value within each customer.

**Approach:**
- Calculate total order value (sum of payment values)
- Use RANK() with PARTITION BY customer_id
- Order by total value descending within each customer

**Key Concepts:**
- Window functions with PARTITION BY
- RANK() for ranking
- Aggregation before window function


In [ ]:
-- Solution 5.1: Order Ranking by Customer
WITH order_values AS (
    SELECT 
        o.CUSTOMER_ID,
        o.ORDER_ID,
        o.ORDER_PURCHASE_TIMESTAMP,
        COALESCE(SUM(op.PAYMENT_VALUE), 0) AS TOTAL_ORDER_VALUE
    FROM CETPA_DB.PUBLIC.orders o
    LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
        ON o.ORDER_ID = op.ORDER_ID
    GROUP BY o.CUSTOMER_ID, o.ORDER_ID, o.ORDER_PURCHASE_TIMESTAMP
)
SELECT 
    CUSTOMER_ID,
    ORDER_ID,
    ORDER_PURCHASE_TIMESTAMP,
    TOTAL_ORDER_VALUE,
    RANK() OVER (
        PARTITION BY CUSTOMER_ID 
        ORDER BY TOTAL_ORDER_VALUE DESC
    ) AS RANK_WITHIN_CUSTOMER
FROM order_values
ORDER BY CUSTOMER_ID, RANK_WITHIN_CUSTOMER;

-- SQL Server Note: Syntax is identical


### Solution 5.2: Running Total by Date

**Problem:** Calculate running total of revenue by order date.

**Approach:**
- First aggregate daily revenue
- Then use SUM() OVER() with ORDER BY date for running total
- Window function calculates cumulative sum

**Key Concepts:**
- Window functions for running totals
- ORDER BY in OVER() clause
- Cumulative calculations


In [ ]:
-- Solution 5.2: Running Total by Date
WITH daily_revenue AS (
    SELECT 
        DATE(o.ORDER_PURCHASE_TIMESTAMP) AS ORDER_DATE,
        COALESCE(SUM(op.PAYMENT_VALUE), 0) AS DAILY_REVENUE
    FROM CETPA_DB.PUBLIC.orders o
    LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
        ON o.ORDER_ID = op.ORDER_ID
    GROUP BY DATE(o.ORDER_PURCHASE_TIMESTAMP)
)
SELECT 
    ORDER_DATE,
    DAILY_REVENUE,
    SUM(DAILY_REVENUE) OVER (
        ORDER BY ORDER_DATE 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS RUNNING_TOTAL
FROM daily_revenue
ORDER BY ORDER_DATE;

-- SQL Server Note: Syntax is identical


### Solution 5.3: Top Product per Category

**Problem:** Find the top-selling product (by quantity) in each category.

**Approach:**
- Aggregate product sales by category
- Use RANK() with PARTITION BY category
- Filter for rank = 1 using QUALIFY (Snowflake) or subquery

**Key Concepts:**
- RANK() with PARTITION BY
- QUALIFY clause (Snowflake-specific)
- Top N per group pattern


In [ ]:
-- Solution 5.3: Top Product per Category (Snowflake - using QUALIFY)
WITH product_sales AS (
    SELECT 
        COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME) AS CATEGORY_NAME_ENGLISH,
        p.PRODUCT_ID,
        p.PRODUCT_CATEGORY_NAME AS CATEGORY_NAME_PORTUGUESE,
        SUM(oi.PRICE + oi.FREIGHT_VALUE) AS TOTAL_QUANTITY_SOLD,
        RANK() OVER (
            PARTITION BY COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME)
            ORDER BY SUM(oi.PRICE + oi.FREIGHT_VALUE) DESC
        ) AS RANK_IN_CATEGORY
    FROM CETPA_DB.PUBLIC.products p
    INNER JOIN CETPA_DB.PUBLIC.order_items oi 
        ON p.PRODUCT_ID = oi.PRODUCT_ID
    LEFT JOIN CETPA_DB.PUBLIC.product_category_name_translation trans 
        ON p.PRODUCT_CATEGORY_NAME = trans.C1
    GROUP BY 
        COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME),
        p.PRODUCT_ID,
        p.PRODUCT_CATEGORY_NAME
)
SELECT 
    CATEGORY_NAME_ENGLISH,
    PRODUCT_ID,
    CATEGORY_NAME_PORTUGUESE,
    TOTAL_QUANTITY_SOLD,
    RANK_IN_CATEGORY
FROM product_sales
QUALIFY RANK_IN_CATEGORY = 1
ORDER BY CATEGORY_NAME_ENGLISH;

-- SQL Server Note: Use subquery instead of QUALIFY
-- SELECT * FROM (
--     SELECT ..., RANK() OVER (...) AS RANK_IN_CATEGORY
--     FROM ...
-- ) ranked
-- WHERE RANK_IN_CATEGORY = 1;


### Solution 5.4: Customer Order Sequence

**Problem:** Show customer orders with sequence numbers and days since previous order.

**Approach:**
- Use ROW_NUMBER() for sequence numbering
- Use LAG() to get previous order date
- Calculate days difference with DATEDIFF

**Key Concepts:**
- ROW_NUMBER() for sequencing
- LAG() for accessing previous row
- Date arithmetic


In [ ]:
-- Solution 5.4: Customer Order Sequence
SELECT 
    o.CUSTOMER_ID,
    o.ORDER_ID,
    o.ORDER_PURCHASE_TIMESTAMP,
    ROW_NUMBER() OVER (
        PARTITION BY o.CUSTOMER_ID 
        ORDER BY o.ORDER_PURCHASE_TIMESTAMP
    ) AS ORDER_SEQUENCE_NUMBER,
    DATEDIFF(
        day,
        LAG(o.ORDER_PURCHASE_TIMESTAMP) OVER (
            PARTITION BY o.CUSTOMER_ID 
            ORDER BY o.ORDER_PURCHASE_TIMESTAMP
        ),
        o.ORDER_PURCHASE_TIMESTAMP
    ) AS DAYS_SINCE_PREVIOUS_ORDER
FROM CETPA_DB.PUBLIC.orders o
ORDER BY o.CUSTOMER_ID, o.ORDER_PURCHASE_TIMESTAMP;

-- SQL Server Note: Syntax is identical


### Solution 5.5: Moving Average Revenue

**Problem:** Calculate 7-day moving average of daily revenue.

**Approach:**
- Aggregate daily revenue first
- Use AVG() OVER() with ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
- This creates a sliding window of 7 days

**Key Concepts:**
- Window frames with ROWS BETWEEN
- Moving averages
- Sliding window calculations


In [ ]:
-- Solution 5.5: Moving Average Revenue
WITH daily_revenue AS (
    SELECT 
        DATE(o.ORDER_PURCHASE_TIMESTAMP) AS ORDER_DATE,
        COALESCE(SUM(op.PAYMENT_VALUE), 0) AS DAILY_REVENUE
    FROM CETPA_DB.PUBLIC.orders o
    LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
        ON o.ORDER_ID = op.ORDER_ID
    GROUP BY DATE(o.ORDER_PURCHASE_TIMESTAMP)
)
SELECT 
    ORDER_DATE,
    DAILY_REVENUE,
    AVG(DAILY_REVENUE) OVER (
        ORDER BY ORDER_DATE 
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS MOVING_AVERAGE_7_DAYS
FROM daily_revenue
ORDER BY ORDER_DATE;

-- SQL Server Note: Syntax is identical


### Solution 5.6: Percent of Total Revenue

**Problem:** Show each order with its percentage of total revenue.

**Approach:**
- Calculate order revenue (sum of payment values)
- Use SUM() OVER() without PARTITION BY to get grand total
- Calculate percentage: (order_revenue / grand_total) * 100

**Key Concepts:**
- Window functions without PARTITION BY
- Percentage calculations
- Grand total calculation


In [ ]:
-- Solution 5.6: Percent of Total Revenue
WITH order_revenue AS (
    SELECT 
        o.ORDER_ID,
        o.ORDER_PURCHASE_TIMESTAMP,
        COALESCE(SUM(op.PAYMENT_VALUE), 0) AS ORDER_REVENUE
    FROM CETPA_DB.PUBLIC.orders o
    LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
        ON o.ORDER_ID = op.ORDER_ID
    GROUP BY o.ORDER_ID, o.ORDER_PURCHASE_TIMESTAMP
)
SELECT 
    ORDER_ID,
    ORDER_PURCHASE_TIMESTAMP,
    ORDER_REVENUE,
    ROUND(
        (ORDER_REVENUE * 100.0 / SUM(ORDER_REVENUE) OVER()), 
        2
    ) AS PERCENT_OF_TOTAL_REVENUE
FROM order_revenue
ORDER BY ORDER_REVENUE DESC;

-- SQL Server Note: Syntax is identical


---

## Section 6: Common Table Expressions (CTEs) (🟡 Intermediate to 🔴 Advanced)

### Solution 6.1: Customer Lifetime Value

**Problem:** Calculate customer lifetime value using CTEs.

**Approach:**
- Use CTE to calculate customer order totals
- Calculate date differences for tenure
- Combine all metrics in final SELECT

**Key Concepts:**
- CTEs for complex calculations
- Date arithmetic
- Customer analytics


In [ ]:
-- Solution 6.1: Customer Lifetime Value
WITH customer_totals AS (
    SELECT 
        c.CUSTOMER_ID,
        c.CUSTOMER_CITY,
        COUNT(DISTINCT o.ORDER_ID) AS TOTAL_ORDERS,
        COALESCE(SUM(op.PAYMENT_VALUE), 0) AS TOTAL_REVENUE,
        MIN(o.ORDER_PURCHASE_TIMESTAMP) AS FIRST_ORDER_DATE,
        MAX(o.ORDER_PURCHASE_TIMESTAMP) AS LAST_ORDER_DATE
    FROM CETPA_DB.PUBLIC.CUSTOMERS c
    LEFT JOIN CETPA_DB.PUBLIC.orders o 
        ON c.CUSTOMER_ID = o.CUSTOMER_ID
    LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
        ON o.ORDER_ID = op.ORDER_ID
    GROUP BY c.CUSTOMER_ID, c.CUSTOMER_CITY
)
SELECT 
    CUSTOMER_ID,
    CUSTOMER_CITY,
    FIRST_ORDER_DATE,
    LAST_ORDER_DATE,
    TOTAL_ORDERS,
    TOTAL_REVENUE,
    CASE 
        WHEN TOTAL_ORDERS > 0 THEN TOTAL_REVENUE / TOTAL_ORDERS
        ELSE 0
    END AS AVERAGE_ORDER_VALUE,
    DATEDIFF(day, FIRST_ORDER_DATE, LAST_ORDER_DATE) AS CUSTOMER_TENURE_DAYS
FROM customer_totals
WHERE TOTAL_ORDERS > 0
ORDER BY TOTAL_REVENUE DESC;

-- SQL Server Note: Syntax is identical


### Solution 6.2: Find Duplicate Orders

**Problem:** Find duplicate orders (same customer, same date, same total payment value).

**Approach:**
- Use CTE to identify duplicate groups
- Join back to show all order IDs in duplicate groups
- Use LISTAGG or STRING_AGG to list order IDs

**Key Concepts:**
- CTEs for duplicate detection
- GROUP BY with HAVING
- String aggregation functions


In [ ]:
-- Solution 6.2: Find Duplicate Orders
WITH order_totals AS (
    SELECT 
        o.CUSTOMER_ID,
        DATE(o.ORDER_PURCHASE_TIMESTAMP) AS ORDER_DATE,
        o.ORDER_ID,
        COALESCE(SUM(op.PAYMENT_VALUE), 0) AS TOTAL_PAYMENT_VALUE
    FROM CETPA_DB.PUBLIC.orders o
    LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
        ON o.ORDER_ID = op.ORDER_ID
    GROUP BY o.CUSTOMER_ID, DATE(o.ORDER_PURCHASE_TIMESTAMP), o.ORDER_ID
),
duplicate_groups AS (
    SELECT 
        CUSTOMER_ID,
        ORDER_DATE,
        TOTAL_PAYMENT_VALUE,
        COUNT(*) AS DUPLICATE_COUNT
    FROM order_totals
    GROUP BY CUSTOMER_ID, ORDER_DATE, TOTAL_PAYMENT_VALUE
    HAVING COUNT(*) > 1
)
SELECT 
    dg.CUSTOMER_ID,
    dg.ORDER_DATE,
    dg.TOTAL_PAYMENT_VALUE,
    LISTAGG(ot.ORDER_ID, ', ') WITHIN GROUP (ORDER BY ot.ORDER_ID) AS ORDER_IDS,
    dg.DUPLICATE_COUNT
FROM duplicate_groups dg
INNER JOIN order_totals ot 
    ON dg.CUSTOMER_ID = ot.CUSTOMER_ID
    AND dg.ORDER_DATE = ot.ORDER_DATE
    AND dg.TOTAL_PAYMENT_VALUE = ot.TOTAL_PAYMENT_VALUE
GROUP BY dg.CUSTOMER_ID, dg.ORDER_DATE, dg.TOTAL_PAYMENT_VALUE, dg.DUPLICATE_COUNT
ORDER BY dg.CUSTOMER_ID, dg.ORDER_DATE;

-- SQL Server Note:
-- Use STRING_AGG instead of LISTAGG:
-- STRING_AGG(ot.ORDER_ID, ', ') WITHIN GROUP (ORDER BY ot.ORDER_ID) AS ORDER_IDS


### Solution 6.3: Product Sales Trend

**Problem:** Show monthly sales trend for each product.

**Approach:**
- First CTE: Aggregate sales by product and month
- Second CTE: Add window functions for previous month and running total
- Calculate month-over-month change

**Key Concepts:**
- Multiple CTEs chaining together
- Window functions in CTEs
- Trend analysis


In [ ]:
-- Solution 6.3: Product Sales Trend
WITH monthly_sales AS (
    SELECT 
        p.PRODUCT_ID,
        EXTRACT(YEAR FROM o.ORDER_PURCHASE_TIMESTAMP) AS YEAR,
        EXTRACT(MONTH FROM o.ORDER_PURCHASE_TIMESTAMP) AS MONTH,
        SUM(oi.PRICE + oi.FREIGHT_VALUE) AS QUANTITY_SOLD_IN_MONTH
    FROM CETPA_DB.PUBLIC.products p
    INNER JOIN CETPA_DB.PUBLIC.order_items oi 
        ON p.PRODUCT_ID = oi.PRODUCT_ID
    INNER JOIN CETPA_DB.PUBLIC.orders o 
        ON oi.ORDER_ID = o.ORDER_ID
    GROUP BY 
        p.PRODUCT_ID,
        EXTRACT(YEAR FROM o.ORDER_PURCHASE_TIMESTAMP),
        EXTRACT(MONTH FROM o.ORDER_PURCHASE_TIMESTAMP)
),
sales_with_trends AS (
    SELECT 
        PRODUCT_ID,
        YEAR || '-' || LPAD(MONTH, 2, '0') AS YEAR_MONTH,
        QUANTITY_SOLD_IN_MONTH,
        LAG(QUANTITY_SOLD_IN_MONTH) OVER (
            PARTITION BY PRODUCT_ID 
            ORDER BY YEAR, MONTH
        ) AS PREVIOUS_MONTH_QUANTITY,
        QUANTITY_SOLD_IN_MONTH - LAG(QUANTITY_SOLD_IN_MONTH) OVER (
            PARTITION BY PRODUCT_ID 
            ORDER BY YEAR, MONTH
        ) AS MONTH_OVER_MONTH_CHANGE,
        SUM(QUANTITY_SOLD_IN_MONTH) OVER (
            PARTITION BY PRODUCT_ID 
            ORDER BY YEAR, MONTH 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS RUNNING_TOTAL_QUANTITY
    FROM monthly_sales
)
SELECT 
    PRODUCT_ID,
    YEAR_MONTH,
    QUANTITY_SOLD_IN_MONTH,
    PREVIOUS_MONTH_QUANTITY,
    MONTH_OVER_MONTH_CHANGE,
    RUNNING_TOTAL_QUANTITY
FROM sales_with_trends
ORDER BY PRODUCT_ID, YEAR_MONTH;

-- SQL Server Note:
-- Use CONCAT or + for string concatenation:
-- CONCAT(YEAR, '-', FORMAT(MONTH, '00')) AS YEAR_MONTH
-- Or: CAST(YEAR AS VARCHAR) + '-' + RIGHT('0' + CAST(MONTH AS VARCHAR), 2)


### Solution 6.4: Customer Segmentation

**Problem:** Segment customers based on order behavior.

**Approach:**
- CTE 1: Calculate customer metrics
- CTE 2: Add segmentation logic with CASE statements
- Calculate days since last order
- Apply business rules for segmentation

**Key Concepts:**
- Customer segmentation
- CASE statements for categorization
- Date calculations
- Business logic implementation


In [ ]:
-- Solution 6.4: Customer Segmentation
WITH customer_metrics AS (
    SELECT 
        c.CUSTOMER_ID,
        COUNT(DISTINCT o.ORDER_ID) AS TOTAL_ORDERS,
        COALESCE(SUM(op.PAYMENT_VALUE), 0) AS TOTAL_REVENUE,
        COALESCE(AVG(op.PAYMENT_VALUE), 0) AS AVERAGE_ORDER_VALUE,
        MAX(o.ORDER_PURCHASE_TIMESTAMP) AS LAST_ORDER_DATE
    FROM CETPA_DB.PUBLIC.CUSTOMERS c
    LEFT JOIN CETPA_DB.PUBLIC.orders o 
        ON c.CUSTOMER_ID = o.CUSTOMER_ID
    LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
        ON o.ORDER_ID = op.ORDER_ID
    GROUP BY c.CUSTOMER_ID
)
SELECT 
    CUSTOMER_ID,
    TOTAL_ORDERS,
    TOTAL_REVENUE,
    AVERAGE_ORDER_VALUE,
    DATEDIFF(day, LAST_ORDER_DATE, CURRENT_DATE()) AS DAYS_SINCE_LAST_ORDER,
    CASE 
        WHEN TOTAL_REVENUE >= 1000 THEN 'VIP'
        WHEN TOTAL_REVENUE >= 500 THEN 'Regular'
        ELSE 'Casual'
    END AS CUSTOMER_SEGMENT,
    CASE 
        WHEN LAST_ORDER_DATE IS NULL THEN 'Never Purchased'
        WHEN DATEDIFF(day, LAST_ORDER_DATE, CURRENT_DATE()) <= 30 THEN 'Active'
        WHEN DATEDIFF(day, LAST_ORDER_DATE, CURRENT_DATE()) <= 90 THEN 'At Risk'
        ELSE 'Churned'
    END AS RECENCY_STATUS
FROM customer_metrics
ORDER BY TOTAL_REVENUE DESC;

-- SQL Server Note:
-- Use GETDATE() instead of CURRENT_DATE():
-- DATEDIFF(day, LAST_ORDER_DATE, GETDATE())


### Solution 6.5: Top 3 Products per Category

**Problem:** Find top 3 products by revenue in each category.

**Approach:**
- CTE: Calculate product revenue
- Use RANK() with PARTITION BY category
- Filter with QUALIFY for top 3

**Key Concepts:**
- Top N per group pattern
- QUALIFY clause (Snowflake)
- Ranking within partitions


In [ ]:
-- Solution 6.5: Top 3 Products per Category
WITH product_revenue AS (
    SELECT 
        COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME) AS CATEGORY_NAME_ENGLISH,
        p.PRODUCT_ID,
        COALESCE(SUM(oi.PRICE + oi.FREIGHT_VALUE), 0) AS TOTAL_REVENUE,
        RANK() OVER (
            PARTITION BY COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME)
            ORDER BY SUM(oi.PRICE + oi.FREIGHT_VALUE) DESC
        ) AS RANK_IN_CATEGORY
    FROM CETPA_DB.PUBLIC.products p
    INNER JOIN CETPA_DB.PUBLIC.order_items oi 
        ON p.PRODUCT_ID = oi.PRODUCT_ID
    LEFT JOIN CETPA_DB.PUBLIC.product_category_name_translation trans 
        ON p.PRODUCT_CATEGORY_NAME = trans.C1
    GROUP BY 
        COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME),
        p.PRODUCT_ID
)
SELECT 
    CATEGORY_NAME_ENGLISH,
    PRODUCT_ID,
    TOTAL_REVENUE,
    RANK_IN_CATEGORY
FROM product_revenue
QUALIFY RANK_IN_CATEGORY <= 3
ORDER BY CATEGORY_NAME_ENGLISH, RANK_IN_CATEGORY;

-- SQL Server Note: Use subquery instead of QUALIFY
-- SELECT * FROM (
--     SELECT ..., RANK() OVER (...) AS RANK_IN_CATEGORY
--     FROM ...
-- ) ranked
-- WHERE RANK_IN_CATEGORY <= 3;


---

## Section 7: Complex Multi-Step Queries (🔴 Advanced)

### Solution 7.1: Order Fulfillment Analysis

**Problem:** Analyze order fulfillment performance.

**Approach:**
- Use CTEs to calculate time differences between order stages
- Aggregate by order status
- Calculate on-time delivery rate
- Handle NULLs in date calculations

**Key Concepts:**
- Complex date arithmetic
- Multiple CTEs for step-by-step calculations
- Percentage calculations
- NULL handling in date operations


In [ ]:
-- Solution 7.1: Order Fulfillment Analysis
WITH order_timings AS (
    SELECT 
        ORDER_STATUS,
        ORDER_ID,
        DATEDIFF(day, ORDER_PURCHASE_TIMESTAMP, ORDER_APPROVED_AT) AS DAYS_TO_APPROVAL,
        DATEDIFF(day, ORDER_APPROVED_AT, ORDER_DELIVERED_CARRIER_DATE) AS DAYS_TO_CARRIER,
        DATEDIFF(day, ORDER_DELIVERED_CARRIER_DATE, ORDER_DELIVERED_CUSTOMER_DATE) AS DAYS_TO_CUSTOMER,
        DATEDIFF(day, ORDER_PURCHASE_TIMESTAMP, ORDER_DELIVERED_CUSTOMER_DATE) AS TOTAL_DELIVERY_TIME,
        CASE 
            WHEN ORDER_DELIVERED_CUSTOMER_DATE <= ORDER_ESTIMATED_DELIVERY_DATE THEN 1
            ELSE 0
        END AS ON_TIME_FLAG
    FROM CETPA_DB.PUBLIC.orders
    WHERE ORDER_APPROVED_AT IS NOT NULL
        AND ORDER_DELIVERED_CARRIER_DATE IS NOT NULL
        AND ORDER_DELIVERED_CUSTOMER_DATE IS NOT NULL
)
SELECT 
    ORDER_STATUS,
    COUNT(*) AS NUMBER_OF_ORDERS,
    AVG(DAYS_TO_APPROVAL) AS AVG_DAYS_TO_APPROVAL,
    AVG(DAYS_TO_CARRIER) AS AVG_DAYS_TO_CARRIER,
    AVG(DAYS_TO_CUSTOMER) AS AVG_DAYS_TO_CUSTOMER,
    AVG(TOTAL_DELIVERY_TIME) AS AVG_TOTAL_DELIVERY_TIME,
    ROUND(AVG(ON_TIME_FLAG) * 100, 2) AS ON_TIME_DELIVERY_RATE_PERCENT
FROM order_timings
GROUP BY ORDER_STATUS
ORDER BY NUMBER_OF_ORDERS DESC;

-- SQL Server Note: Syntax is identical


### Solution 7.2: Review Score Impact on Revenue

**Problem:** Analyze relationship between review scores and revenue.

**Approach:**
- Join order_reviews with orders and order_payments
- Aggregate by review score
- Calculate percentages using window functions

**Key Concepts:**
- Joining review data with order data
- Aggregation by review score
- Percentage calculations


In [ ]:
-- Solution 7.2: Review Score Impact on Revenue
WITH review_revenue AS (
    SELECT 
        orv.REVIEW_SCORE,
        COUNT(DISTINCT orv.ORDER_ID) AS NUMBER_OF_REVIEWS,
        AVG(op.PAYMENT_VALUE) AS AVERAGE_ORDER_VALUE,
        SUM(op.PAYMENT_VALUE) AS TOTAL_REVENUE
    FROM CETPA_DB.PUBLIC.order_reviews orv
    INNER JOIN CETPA_DB.PUBLIC.orders o 
        ON orv.ORDER_ID = o.ORDER_ID
    INNER JOIN CETPA_DB.PUBLIC.order_payments op 
        ON o.ORDER_ID = op.ORDER_ID
    WHERE orv.REVIEW_SCORE IS NOT NULL
    GROUP BY orv.REVIEW_SCORE
)
SELECT 
    REVIEW_SCORE,
    NUMBER_OF_REVIEWS,
    ROUND(AVERAGE_ORDER_VALUE, 2) AS AVERAGE_ORDER_VALUE,
    TOTAL_REVENUE,
    ROUND(
        (NUMBER_OF_REVIEWS * 100.0 / SUM(NUMBER_OF_REVIEWS) OVER()), 
        2
    ) AS PERCENTAGE_OF_TOTAL_REVIEWS
FROM review_revenue
ORDER BY REVIEW_SCORE DESC;

-- SQL Server Note: Syntax is identical


### Solution 7.3: Geographic Sales Analysis

**Problem:** Analyze sales by geographic region.

**Approach:**
- Use multiple CTEs to join customers, sellers, orders, and products
- Aggregate by state
- Use window functions to find top category per state

**Key Concepts:**
- Complex multi-table joins
- Geographic analysis
- Top N per group with window functions


In [ ]:
-- Solution 7.3: Geographic Sales Analysis
WITH state_customers AS (
    SELECT 
        CUSTOMER_STATE AS STATE,
        COUNT(DISTINCT CUSTOMER_ID) AS NUMBER_OF_CUSTOMERS
    FROM CETPA_DB.PUBLIC.CUSTOMERS
    WHERE CUSTOMER_STATE IS NOT NULL
    GROUP BY CUSTOMER_STATE
),
state_sellers AS (
    SELECT 
        SELLER_STATE AS STATE,
        COUNT(DISTINCT SELLER_ID) AS NUMBER_OF_SELLERS
    FROM CETPA_DB.PUBLIC.sellers
    WHERE SELLER_STATE IS NOT NULL
    GROUP BY SELLER_STATE
),
state_orders AS (
    SELECT 
        c.CUSTOMER_STATE AS STATE,
        COUNT(DISTINCT o.ORDER_ID) AS NUMBER_OF_ORDERS,
        COALESCE(SUM(op.PAYMENT_VALUE), 0) AS TOTAL_REVENUE,
        COALESCE(AVG(op.PAYMENT_VALUE), 0) AS AVERAGE_ORDER_VALUE
    FROM CETPA_DB.PUBLIC.orders o
    INNER JOIN CETPA_DB.PUBLIC.CUSTOMERS c 
        ON o.CUSTOMER_ID = c.CUSTOMER_ID
    LEFT JOIN CETPA_DB.PUBLIC.order_payments op 
        ON o.ORDER_ID = op.ORDER_ID
    WHERE c.CUSTOMER_STATE IS NOT NULL
    GROUP BY c.CUSTOMER_STATE
),
state_categories AS (
    SELECT 
        c.CUSTOMER_STATE AS STATE,
        COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME) AS CATEGORY_NAME,
        SUM(oi.PRICE + oi.FREIGHT_VALUE) AS CATEGORY_REVENUE,
        RANK() OVER (
            PARTITION BY c.CUSTOMER_STATE 
            ORDER BY SUM(oi.PRICE + oi.FREIGHT_VALUE) DESC
        ) AS CATEGORY_RANK
    FROM CETPA_DB.PUBLIC.orders o
    INNER JOIN CETPA_DB.PUBLIC.CUSTOMERS c 
        ON o.CUSTOMER_ID = c.CUSTOMER_ID
    INNER JOIN CETPA_DB.PUBLIC.order_items oi 
        ON o.ORDER_ID = oi.ORDER_ID
    INNER JOIN CETPA_DB.PUBLIC.products p 
        ON oi.PRODUCT_ID = p.PRODUCT_ID
    LEFT JOIN CETPA_DB.PUBLIC.product_category_name_translation trans 
        ON p.PRODUCT_CATEGORY_NAME = trans.C1
    WHERE c.CUSTOMER_STATE IS NOT NULL
    GROUP BY c.CUSTOMER_STATE, COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME)
)
SELECT 
    COALESCE(sc.STATE, ss.STATE, so.STATE) AS STATE,
    COALESCE(sc.NUMBER_OF_CUSTOMERS, 0) AS NUMBER_OF_CUSTOMERS,
    COALESCE(ss.NUMBER_OF_SELLERS, 0) AS NUMBER_OF_SELLERS,
    COALESCE(so.NUMBER_OF_ORDERS, 0) AS NUMBER_OF_ORDERS,
    COALESCE(so.TOTAL_REVENUE, 0) AS TOTAL_REVENUE,
    COALESCE(so.AVERAGE_ORDER_VALUE, 0) AS AVERAGE_ORDER_VALUE,
    MAX(CASE WHEN stc.CATEGORY_RANK = 1 THEN stc.CATEGORY_NAME END) AS TOP_PRODUCT_CATEGORY
FROM state_customers sc
FULL OUTER JOIN state_sellers ss 
    ON sc.STATE = ss.STATE
FULL OUTER JOIN state_orders so 
    ON COALESCE(sc.STATE, ss.STATE) = so.STATE
LEFT JOIN state_categories stc 
    ON COALESCE(sc.STATE, ss.STATE, so.STATE) = stc.STATE
GROUP BY 
    COALESCE(sc.STATE, ss.STATE, so.STATE),
    COALESCE(sc.NUMBER_OF_CUSTOMERS, 0),
    COALESCE(ss.NUMBER_OF_SELLERS, 0),
    COALESCE(so.NUMBER_OF_ORDERS, 0),
    COALESCE(so.TOTAL_REVENUE, 0),
    COALESCE(so.AVERAGE_ORDER_VALUE, 0)
ORDER BY TOTAL_REVENUE DESC;

-- SQL Server Note: Syntax is identical


In [ ]:
-- Solution 7.4: Payment Installment Analysis
WITH payment_stats AS (
    SELECT 
        PAYMENT_TYPE,
        AVG(PAYMENT_INSTALLMENTS) AS AVERAGE_INSTALLMENTS,
        COUNT(*) AS TOTAL_NUMBER_OF_PAYMENTS,
        SUM(PAYMENT_VALUE) AS TOTAL_PAYMENT_VALUE,
        AVG(PAYMENT_VALUE) AS AVERAGE_PAYMENT_VALUE,
        COUNT(DISTINCT ORDER_ID) AS NUMBER_OF_UNIQUE_ORDERS
    FROM CETPA_DB.PUBLIC.order_payments
    WHERE PAYMENT_TYPE IS NOT NULL
    GROUP BY PAYMENT_TYPE
)
SELECT 
    PAYMENT_TYPE,
    ROUND(AVERAGE_INSTALLMENTS, 2) AS AVERAGE_INSTALLMENTS,
    TOTAL_NUMBER_OF_PAYMENTS,
    TOTAL_PAYMENT_VALUE,
    ROUND(AVERAGE_PAYMENT_VALUE, 2) AS AVERAGE_PAYMENT_VALUE,
    ROUND(
        (NUMBER_OF_UNIQUE_ORDERS * 100.0 / SUM(NUMBER_OF_UNIQUE_ORDERS) OVER()), 
        2
    ) AS PERCENTAGE_OF_ORDERS
FROM payment_stats
ORDER BY TOTAL_PAYMENT_VALUE DESC;

-- SQL Server Note: Syntax is identical


### Solution 7.5: Product Recommendation Score

**Problem:** Create product recommendation score based on multiple factors.

**Approach:**
- CTE 1: Calculate quantity sold and revenue rank
- CTE 2: Calculate average review score
- CTE 3: Combine all factors into final score
- Order by score descending

**Key Concepts:**
- Complex scoring algorithms
- Multiple CTEs for different calculations
- Weighted scoring


In [ ]:
-- Solution 7.5: Product Recommendation Score
WITH product_sales AS (
    SELECT 
        p.PRODUCT_ID,
        COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME) AS CATEGORY_NAME,
        SUM(oi.PRICE + oi.FREIGHT_VALUE) AS TOTAL_QUANTITY_SOLD,
        RANK() OVER (
            PARTITION BY COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME)
            ORDER BY SUM(oi.PRICE + oi.FREIGHT_VALUE) DESC
        ) AS REVENUE_RANK_IN_CATEGORY
    FROM CETPA_DB.PUBLIC.products p
    INNER JOIN CETPA_DB.PUBLIC.order_items oi 
        ON p.PRODUCT_ID = oi.PRODUCT_ID
    LEFT JOIN CETPA_DB.PUBLIC.product_category_name_translation trans 
        ON p.PRODUCT_CATEGORY_NAME = trans.C1
    GROUP BY p.PRODUCT_ID, COALESCE(trans.C2, p.PRODUCT_CATEGORY_NAME)
),
product_reviews AS (
    SELECT 
        oi.PRODUCT_ID,
        AVG(orv.REVIEW_SCORE) AS AVERAGE_REVIEW_SCORE,
        COUNT(DISTINCT orv.REVIEW_ID) AS NUMBER_OF_REVIEWS
    FROM CETPA_DB.PUBLIC.order_items oi
    INNER JOIN CETPA_DB.PUBLIC.order_reviews orv 
        ON oi.ORDER_ID = orv.ORDER_ID
    WHERE orv.REVIEW_SCORE IS NOT NULL
    GROUP BY oi.PRODUCT_ID
),
product_scores AS (
    SELECT 
        ps.PRODUCT_ID,
        ps.CATEGORY_NAME,
        ps.TOTAL_QUANTITY_SOLD,
        COALESCE(pr.AVERAGE_REVIEW_SCORE, 0) AS AVERAGE_REVIEW_SCORE,
        COALESCE(pr.NUMBER_OF_REVIEWS, 0) AS NUMBER_OF_REVIEWS,
        ps.REVENUE_RANK_IN_CATEGORY,
        -- Calculate rank score (inverse rank, normalized)
        (1.0 / ps.REVENUE_RANK_IN_CATEGORY) * 100 AS CATEGORY_RANK_SCORE,
        -- Final Score Calculation
        (ps.TOTAL_QUANTITY_SOLD * 0.4) + 
        (COALESCE(pr.AVERAGE_REVIEW_SCORE, 0) * 20 * 0.3) + 
        ((1.0 / ps.REVENUE_RANK_IN_CATEGORY) * 100 * 0.3) AS FINAL_SCORE
    FROM product_sales ps
    LEFT JOIN product_reviews pr 
        ON ps.PRODUCT_ID = pr.PRODUCT_ID
)
SELECT 
    PRODUCT_ID,
    CATEGORY_NAME,
    TOTAL_QUANTITY_SOLD,
    ROUND(AVERAGE_REVIEW_SCORE, 2) AS AVERAGE_REVIEW_SCORE,
    NUMBER_OF_REVIEWS,
    REVENUE_RANK_IN_CATEGORY,
    ROUND(FINAL_SCORE, 2) AS FINAL_SCORE
FROM product_scores
ORDER BY FINAL_SCORE DESC
LIMIT 20;

-- SQL Server Note:
-- Use TOP instead of LIMIT:
-- SELECT TOP 20 ...


---

## Section 8: Data Quality and Troubleshooting (🟡 Intermediate)

### Solution 8.1: Data Quality Check

**Problem:** Create a data quality report for critical columns.

**Approach:**
- Use UNION ALL to combine results from multiple tables
- Calculate NULL counts and percentages
- Use CASE for status classification

**Key Concepts:**
- Data quality monitoring
- UNION ALL for combining results
- Percentage calculations
- Status classification


In [ ]:
-- Solution 8.1: Data Quality Check
WITH quality_checks AS (
    SELECT 
        'orders' AS TABLE_NAME,
        'CUSTOMER_ID' AS COLUMN_NAME,
        COUNT(*) AS TOTAL_ROWS,
        COUNT(*) - COUNT(CUSTOMER_ID) AS NULL_COUNT
    FROM CETPA_DB.PUBLIC.orders
    UNION ALL
    SELECT 
        'order_items',
        'PRODUCT_ID',
        COUNT(*),
        COUNT(*) - COUNT(PRODUCT_ID)
    FROM CETPA_DB.PUBLIC.order_items
    UNION ALL
    SELECT 
        'order_payments',
        'ORDER_ID',
        COUNT(*),
        COUNT(*) - COUNT(ORDER_ID)
    FROM CETPA_DB.PUBLIC.order_payments
)
SELECT 
    TABLE_NAME,
    COLUMN_NAME,
    TOTAL_ROWS,
    NULL_COUNT,
    ROUND((NULL_COUNT * 100.0 / TOTAL_ROWS), 2) AS NULL_PERCENTAGE,
    CASE 
        WHEN NULL_COUNT = 0 THEN '✅ PASS'
        WHEN (NULL_COUNT * 100.0 / TOTAL_ROWS) < 5 THEN '⚠️ WARNING'
        ELSE '🔴 FAIL'
    END AS DATA_QUALITY_STATUS
FROM quality_checks
ORDER BY NULL_PERCENTAGE DESC;

-- SQL Server Note: Syntax is identical


### Solution 8.2: Find Orphaned Records

**Problem:** Find orphaned records across multiple tables.

**Approach:**
- Use LEFT JOINs to find records without valid foreign keys
- Count orphaned records for each scenario
- Use UNION ALL to combine results

**Key Concepts:**
- Data integrity checks
- LEFT JOIN + WHERE NULL pattern
- Orphaned record detection


In [ ]:
-- Solution 8.2: Find Orphaned Records
WITH orphaned_checks AS (
    SELECT 
        'Order Items without valid Order ID' AS SCENARIO,
        COUNT(*) AS ORPHANED_COUNT
    FROM CETPA_DB.PUBLIC.order_items oi
    LEFT JOIN CETPA_DB.PUBLIC.orders o 
        ON oi.ORDER_ID = o.ORDER_ID
    WHERE o.ORDER_ID IS NULL
    UNION ALL
    SELECT 
        'Order Payments without valid Order ID',
        COUNT(*)
    FROM CETPA_DB.PUBLIC.order_payments op
    LEFT JOIN CETPA_DB.PUBLIC.orders o 
        ON op.ORDER_ID = o.ORDER_ID
    WHERE o.ORDER_ID IS NULL
    UNION ALL
    SELECT 
        'Order Reviews without valid Order ID',
        COUNT(*)
    FROM CETPA_DB.PUBLIC.order_reviews orv
    LEFT JOIN CETPA_DB.PUBLIC.orders o 
        ON orv.ORDER_ID = o.ORDER_ID
    WHERE o.ORDER_ID IS NULL
    UNION ALL
    SELECT 
        'Orders without valid Customer ID',
        COUNT(*)
    FROM CETPA_DB.PUBLIC.orders o
    LEFT JOIN CETPA_DB.PUBLIC.CUSTOMERS c 
        ON o.CUSTOMER_ID = c.CUSTOMER_ID
    WHERE c.CUSTOMER_ID IS NULL
)
SELECT 
    SCENARIO,
    ORPHANED_COUNT,
    CASE 
        WHEN ORPHANED_COUNT = 0 THEN '✅ No Issues'
        ELSE '🔴 Data Quality Issue'
    END AS STATUS
FROM orphaned_checks
ORDER BY ORPHANED_COUNT DESC;

-- SQL Server Note: Syntax is identical


### Solution 8.3: Inconsistent Data Detection

**Problem:** Detect various data inconsistencies.

**Approach:**
- Use CTEs to calculate expected values
- Compare expected vs actual values
- Use CASE to identify inconsistency types
- UNION ALL to combine different checks

**Key Concepts:**
- Data validation
- Expected vs actual comparisons
- Multiple inconsistency checks


In [ ]:
-- Solution 8.3: Inconsistent Data Detection
WITH order_item_totals AS (
    SELECT 
        oi.ORDER_ID,
        SUM(COALESCE(oi.PRICE, 0) + COALESCE(oi.FREIGHT_VALUE, 0)) AS EXPECTED_TOTAL
    FROM CETPA_DB.PUBLIC.order_items oi
    GROUP BY oi.ORDER_ID
),
order_payment_totals AS (
    SELECT 
        op.ORDER_ID,
        SUM(COALESCE(op.PAYMENT_VALUE, 0)) AS ACTUAL_PAYMENT_TOTAL
    FROM CETPA_DB.PUBLIC.order_payments op
    GROUP BY op.ORDER_ID
),
inconsistencies AS (
    SELECT 
        o.ORDER_ID,
        'Payment value does not match order items total' AS INCONSISTENCY_TYPE
    FROM CETPA_DB.PUBLIC.orders o
    INNER JOIN order_item_totals oit 
        ON o.ORDER_ID = oit.ORDER_ID
    LEFT JOIN order_payment_totals opt 
        ON o.ORDER_ID = opt.ORDER_ID
    WHERE ABS(COALESCE(opt.ACTUAL_PAYMENT_TOTAL, 0) - oit.EXPECTED_TOTAL) > 0.01
    UNION ALL
    SELECT 
        o.ORDER_ID,
        'Order has payment but no order items'
    FROM CETPA_DB.PUBLIC.orders o
    INNER JOIN order_payment_totals opt 
        ON o.ORDER_ID = opt.ORDER_ID
    LEFT JOIN order_item_totals oit 
        ON o.ORDER_ID = oit.ORDER_ID
    WHERE oit.ORDER_ID IS NULL
    UNION ALL
    SELECT 
        o.ORDER_ID,
        'Order has order items but no payment'
    FROM CETPA_DB.PUBLIC.orders o
    INNER JOIN order_item_totals oit 
        ON o.ORDER_ID = oit.ORDER_ID
    LEFT JOIN order_payment_totals opt 
        ON o.ORDER_ID = opt.ORDER_ID
    WHERE opt.ORDER_ID IS NULL
    UNION ALL
    SELECT 
        o.ORDER_ID,
        'Order delivered before being approved'
    FROM CETPA_DB.PUBLIC.orders o
    WHERE o.ORDER_DELIVERED_CUSTOMER_DATE IS NOT NULL
        AND o.ORDER_APPROVED_AT IS NOT NULL
        AND o.ORDER_DELIVERED_CUSTOMER_DATE < o.ORDER_APPROVED_AT
)
SELECT 
    ORDER_ID,
    INCONSISTENCY_TYPE
FROM inconsistencies
ORDER BY ORDER_ID, INCONSISTENCY_TYPE;

-- SQL Server Note: Syntax is identical


---

## Summary

This notebook contains complete solutions to all SQL exercise questions covering:

✅ **Basic Joins** - LEFT JOIN, INNER JOIN patterns  
✅ **Multiple Joins** - Complex multi-table relationships  
✅ **NULL Handling** - COALESCE, safe calculations  
✅ **GROUP BY** - Aggregations and HAVING  
✅ **Window Functions** - RANK, ROW_NUMBER, running totals, moving averages  
✅ **CTEs** - Complex multi-step queries, duplicate detection  
✅ **Advanced Queries** - Multi-CTE solutions, scoring algorithms  
✅ **Data Quality** - Validation, orphaned records, inconsistencies  

**Key Takeaways:**

1. **Always handle NULLs** - Use COALESCE/NVL to prevent calculation errors
2. **Choose the right join** - LEFT JOIN preserves all rows, INNER JOIN only matches
3. **Use CTEs for complexity** - Break complex queries into logical steps
4. **Window functions are powerful** - Use for rankings, running totals, and comparisons
5. **Test incrementally** - Build queries step by step, testing each addition
6. **Document your logic** - Add comments for complex calculations

**SQL Server Differences:**
- Use `TOP N` instead of `LIMIT N`
- Use `STRING_AGG` instead of `LISTAGG`
- Use `GETDATE()` instead of `CURRENT_DATE()`
- Use `YEAR()` and `MONTH()` instead of `EXTRACT()`
- Use subqueries instead of `QUALIFY` clause
- String concatenation uses `+` instead of `||`

**Practice Tips:**
- Start with simple queries and add complexity gradually
- Test with sample data before running on full dataset
- Verify NULL handling in all calculations
- Use CTEs to make complex queries readable
- Always validate your results make business sense

---


